In [3]:
from utils import read_json
from utils import (get_model_openAI,get_model_groq,get_openAI_embeddings)

In [4]:
keys = read_json("keys.json")

In [5]:
# Initialize the OpenAI chat model
llm = get_model_openAI(model = "gpt-3.5-turbo")

#### 01 

In [7]:

from langchain.agents import Tool,initialize_agent, AgentType
from langchain_community.tools.searchapi.tool import SearchApiAPIWrapper

search = SearchApiAPIWrapper(searchapi_api_key= keys.get("Keys").get("SEARCh_API_KEY"))

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="Useful for answering questions about current events or general knowledge."
    )
]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

response = agent.run("What's the latest news about AI in 2025?")
print(response)




> Entering new AgentExecutor chain...
I should use the Search tool to find the latest news about AI in 2025.
Action: Search
Action Input: "latest news about AI in 2025"
Observation: Summary: Chinese tech giant Baidu has announced plans to make its next-generation artificial intelligence model, Ernie, open-source by June 30, ...
At I/O 2025, our CEO Sundar Pichai shared how decades of AI research are now becoming reality for people, businesses and communities all over ...
Generative AI saw particularly strong momentum, attracting $33.9 billion globally in private investment—an 18.7% increase from 2023. AI business usage is also ...
In 2025, AI will evolve from a tool for work and home to an integral part of both. AI-powered agents will do more with greater autonomy and help simplify your ...
Anthropic tests AI running a real business with bizarre results · Applications. June 27, 2025 ; Nvidia reclaims title of most valuable company on AI momentum.
In 2025, technology companies are foc

In [12]:
# Custom Tool 
    # A custom implementation of the llm-math tool (for math evaluation)
    # A custom implementation of the wikipedia tool (for fetching summaries)


import os
import wikipedia
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain.tools import tool

llm = get_model_openAI(model = "gpt-3.5-turbo")

# -------------------------------
# Custom Tool: Math Evaluator
# -------------------------------
@tool
def calculate(expression: str) -> str:
    """Evaluates a basic math expression."""
    try:
        result = eval(expression)
        return f"The result of {expression} is {result}."
    except Exception as e:
        return f"Error evaluating expression: {e}"

# -------------------------------
# Custom Tool: Wikipedia Search
# -------------------------------
@tool
def wiki_search(query: str) -> str:
    """Searches Wikipedia and returns a summary of the topic."""
    try:
        summary = wikipedia.summary(query, sentences=2)
        return summary
    except Exception as e:
        return f"Error fetching Wikipedia summary: {e}"

# -------------------------------
# Initialize LLM and Agent
# -------------------------------


tools = [
    Tool.from_function(
        func=calculate,
        name="Calculator",
        description="Evaluates basic math expressions like 'sqrt(256)' or '2 + 2'."
    ),
    Tool.from_function(
        func=wiki_search,
        name="Wikipedia",
        description="Searches Wikipedia and returns a short summary of a topic."
    )
]


agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)

# -------------------------------
# Run the Agent
# -------------------------------
query = "What is the square root of 256, and who discovered gravity?"
response = agent.run(query)

print("\nFinal Answer:\n", response)




> Entering new AgentExecutor chain...
I can use the Calculator tool to find the square root of 256 and the Wikipedia tool to find out who discovered gravity.
Action: Calculator
Action Input: sqrt(256)
Observation: Error evaluating expression: name 'sqrt' is not defined
Thought:It seems like the Calculator tool doesn't recognize the sqrt function. I'll try using a different expression to calculate the square root of 256. Let's also search for the discovery of gravity on Wikipedia.
Action: Calculator
Action Input: 256^(1/2)
Observation: Error evaluating expression: unsupported operand type(s) for ^: 'int' and 'float'
Thought:The Calculator tool doesn't support the ^ operator for exponentiation. I'll use a different approach to find the square root of 256 and search for the discovery of gravity on Wikipedia.
Action: Wikipedia
Action Input: Discovery of gravity
Observation: The Gravity Discovery Centre and Observatory is a "hands-on" science education, astronomy, Aboriginal culture and t

In [18]:
from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_types import AgentType
from langchain.llms import OpenAI

llm = get_model_openAI(model = "gpt-3.5-turbo")

# Define a simple function
def get_weather(location: str) -> str:
    return f"The weather in {location} is sunny."

# Wrap it as a Tool
weather_tool = Tool(
    name="GetWeather",
    func=get_weather,
    description="Use this tool to get the weather for a given location."
)

# Initialize tools list
tools = []

# Append the tool
tools.append(weather_tool)


# Define another function
def basic_calculator(expression: str) -> str:
    try:
        result = eval(expression)
        return f"The result of '{expression}' is {result}."
    except Exception as e:
        return f"Error evaluating expression: {str(e)}"

# Wrap it as a Tool
calculator_tool = Tool(
    name="BasicCalculator",
    func=basic_calculator,
    description="Use this tool to evaluate basic math expressions like '2 + 2' or '10 / 3'."
)

# Append the new tool
tools.append(calculator_tool)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


# Example query to test the agent
response = agent.run("What's the weather in Bangalore and what is 15 * 3?")

print(response)


# tools list contains both:

    # GetWeather – for weather info
    # BasicCalculator – for evaluating math expressions

# When to Use tools.append()
    # When you're dynamically building a list of tools.
    # When you want to conditionally add tools based on some logic.
    # When you're modularizing your tool definitions.



> Entering new AgentExecutor chain...
I need to find out the weather in Bangalore and also calculate 15 * 3.
Action: GetWeather
Action Input: Bangalore
Observation: The weather in Bangalore is sunny.
Thought:I have the weather information, now I need to calculate 15 * 3.
Action: BasicCalculator
Action Input: 15 * 3
Observation: The result of '15 * 3' is 45.
Thought:I have the weather in Bangalore (sunny) and the result of 15 * 3 (45).
Final Answer: The weather in Bangalore is sunny and 15 * 3 is 45.

> Finished chain.
The weather in Bangalore is sunny and 15 * 3 is 45.


In [22]:
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b

tools = [add, multiply]

llm = get_model_openAI(model="gpt-4")
llm_with_tools = llm.bind_tools(tools)

response = llm_with_tools.invoke(
    "Whats 119 times 8 minus 20. Don't do any math yourself, only use tools for math. Respect order of operations")

print(response)

# Use bind_tools() when:

    # You're working with LangChain Expression Language (LCEL).
    # You want to integrate tools directly into the LLM's reasoning.
    # You want to avoid using agents but still allow tool usage.

content='' additional_kwargs={'tool_calls': [{'id': 'call_Ln4autqyhqzZPhp4hTsV4x0E', 'function': {'arguments': '{\n  "a": 119,\n  "b": 8\n}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 99, 'total_tokens': 120, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-BooMUB7doHwj79v74oapQyqe3wRma', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--ad4c4ceb-98a5-4366-ad54-1e2a5c6c381a-0' tool_calls=[{'name': 'multiply', 'args': {'a': 119, 'b': 8}, 'id': 'call_Ln4autqyhqzZPhp4hTsV4x0E', 'type': 'tool_call'}] usage_metadata={'input_tokens': 99, 'output_tokens': 21, 'total_tokens': 120, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 